In [7]:
# import keras.callbacks
from music21 import *

import re
import xml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dense, Dropout
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [10]:
with open('environment.txt') as f:
    env = f.readlines()
# you may also want to remove whitespace characters like \n at the end of each line
env = [x.strip() for x in env]

us = environment.UserSettings()
us['musicxmlPath'] = env[0]
us['musescoreDirectPNGPath'] = env[0]



In [11]:
# fix random seed for reproducibility
np.random.seed(7)

In [12]:
bachRegex = re.compile(r'.*bwv.*', re.IGNORECASE)
bd = corpus.search(bachRegex)

In [13]:
def convertSongToDataFrame(song):
    df = pd.DataFrame()
    
    currentKeySignature = None
    currentTimeSignature = None
    for measure in song.measures(0, None, collect=('TimeSignature','KeySignature')):
        if measure.keySignature:
            currentKeySignature = measure.keySignature
        if measure.timeSignature:
            currentTimeSignature = measure.timeSignature.ratioString
        for i in measure.iter:
            if type(i) is chord.Chord:
                df2 = pd.DataFrame([currentKeySignature, currentTimeSignature, roman.romanNumeralFromChord(i, currentKeySignature).figure, i.duration.quarterLength, i.offset + measure.offset])
                df = df.append(df2[0], ignore_index = True)
    return df


In [14]:
# only run if you want to create csv files
import os.path
import os
if not os.path.exists('./data'):
    os.makedirs("./data")
for s in bd:
    fname = "./data/" + s.sourcePath.split("/")[1]+'.csv'
    if os.path.isfile(fname):
        continue
    df = convertSongToDataFrame(s.parse().chordify())
    df.to_csv(fname, sep=',')
    
#failed at bwv276.mxl

SyntaxError: invalid syntax (<ipython-input-14-63e9a70184f8>, line 1)

In [15]:

scores = []
import glob, os
os.chdir("C:/Users/errpo/Desktop/projects/project folder/music_prediction/data")
for file in glob.glob("*.csv"):
    scores.append(file)
scoreDf = []

for i in scores:
    score = pd.read_csv(i,usecols=[1,2,3,4,5])
    scoreDf.append(score)
    
# print(scoreDf[1])
scores = pd.concat(scoreDf)

keySignatures = pd.DataFrame([scores['0'].astype('category').cat.codes.unique()], columns=scores['0'].unique())
timeSignatures = pd.DataFrame([scores['1'].astype('category').cat.codes.unique()], columns=scores['1'].unique())
chords = pd.DataFrame([scores['2'].astype('category').cat.codes.unique()], columns=scores['2'].unique())

scores['0'] = scores['0'].astype('category').cat.codes
scores['1'] = scores['1'].astype('category').cat.codes
scores['2'] = scores['2'].astype('category').cat.codes
print(scores)

     0  1    2     3       4
0    7  4   99  1.00    0.00
1    7  4  329  0.50    1.00
2    7  4  329  0.50    1.50
3    7  4   99  0.50    2.00
4    7  4   99  0.50    2.50
5    7  4  774  0.50    3.00
6    7  4  774  0.50    3.50
7    7  4  122  0.50    4.00
8    7  4  122  0.50    4.50
9    7  4  237  0.25    5.00
10   7  4  237  0.25    5.25
11   7  4  291  0.25    5.50
12   7  4  599  0.25    5.75
13   7  4  262  0.50    6.00
14   7  4  851  0.50    6.50
15   7  4   99  1.00    7.00
16   7  4   99  1.00    8.00
17   7  4  237  0.50    9.00
18   7  4  583  0.25    9.50
19   7  4  273  0.25    9.75
20   7  4  895  0.50   10.00
21   7  4  321  0.25   10.50
22   7  4  886  0.25   10.75
23   7  4  122  0.50   11.00
24   7  4  163  0.50   11.50
25   7  4  305  0.50   12.00
26   7  4  305  0.50   12.50
27   7  4  813  0.50   13.00
28   7  4   47  0.25   13.50
29   7  4  832  0.25   13.75
..  .. ..  ...   ...     ...
193  3  4  441  0.25  102.75
194  3  4  401  0.50  103.00
195  3  4  540

In [16]:
# pd.read_csv('bwv1.6.mxl.csv')

In [17]:
#LSTM
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

In [18]:
data = scores.values
examples = 100
y_examples = 10
nb_samples = len(data) - examples - y_examples


In [19]:
input_list = [np.expand_dims(np.atleast_2d(data[i:examples+i,:]), axis=0) for i in range(nb_samples)]
input_mat = np.concatenate(input_list, axis=0)


In [20]:
target_list = [np.atleast_2d(data[i+examples:examples+i+y_examples,0]) for i in range(nb_samples)]
target_mat = np.concatenate(target_list, axis=0)


In [21]:
trials = input_mat.shape[0]
features = input_mat.shape[2]
hidden = 64
model = Sequential()
model.add(LSTM(hidden, input_shape=(examples, features)))
model.add(Dropout(.2))
# model.add(Flatten())
model.add(Dense(y_examples))
model.add(Activation('linear'))
model.compile(loss='mse', optimizer='rmsprop')

In [22]:
history = LossHistory()
model = model.fit(input_mat, target_mat, nb_epoch=100, batch_size=400, callbacks=[history])

c:\users\errpo\anaconda3\envs\music\lib\site-packages\keras\models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/100
17575/17575 [==============================] - 26s 1ms/step - loss: 49.8768
Epoch 2/100
17575/17575 [==============================] - 26s 1ms/step - loss: 25.3969
Epoch 3/100
17575/17575 [==============================] - 22s 1ms/step - loss: 16.6499
Epoch 4/100
17575/17575 [==============================] - 20s 1ms/step - loss: 11.8401
Epoch 5/100
17575/17575 [==============================] - 19s 1ms/step - loss: 7.6324
Epoch 6/100
17575/17575 [==============================] - 18s 1ms/step - loss: 5.6216
Epoch 7/100
17575/17575 [==============================] - 18s 1ms/step - loss: 4.5575
Epoch 8/100
17575/17575 [==============================] - 20s 1ms/step - loss: 3.9406
Epoch 9/100
17575/17575 [==============================] - 18s 1ms/step - loss: 3.5756
Epoch 10/100
17575/17575 [==============================] - 26s 1ms/step - loss: 3.3802
Epoch 11/100
17575/17575 [==============================] - 21s 1ms/step - loss: 3.1915
Epoch 12/100
17575/17575 [===========

Epoch 94/100
17575/17575 [==============================] - 18s 1ms/step - loss: 1.7300
Epoch 95/100
17575/17575 [==============================] - 21s 1ms/step - loss: 1.7349
Epoch 96/100
17575/17575 [==============================] - 18s 1ms/step - loss: 1.7580
Epoch 97/100
17575/17575 [==============================] - 19s 1ms/step - loss: 1.7461
Epoch 98/100
17575/17575 [==============================] - 19s 1ms/step - loss: 1.7401
Epoch 99/100
17575/17575 [==============================] - 19s 1ms/step - loss: 1.6909
Epoch 100/100
17575/17575 [==============================] - 19s 1ms/step - loss: 1.7195


AttributeError: 'History' object has no attribute 'predict'